In [1]:
import numpy as np
import scipy.sparse as sp

from label_abstraction.mdp import *

### Define two label MDPs

In [2]:
# state 1: init
# state 2: contains sample
# state 3: no sample
T0 = np.eye(3)
T1 = np.array([[0,0.5,0.5], [0,1,0], [0,0,1]])

def output_fcn(n):
    if n == 0:
        return 'init'    # label unknown
    if n == 1:
        return 'safe'    # can traverse region
    if n == 2:
        return 'unsafe'  # can not traverse region
    
map1 = MDP([T0, T1], input_fcn=lambda meas1: meas1, input_name='meas1',
                     output_fcn=output_fcn, output_name='label1')

map2 = MDP([T0, T1], input_fcn=lambda meas2: meas2, input_name='meas2',
                     output_fcn=output_fcn, output_name='label2')

### Define gridworld system MDP

In [3]:
# gridworld mdp
l_x = 4  # x length
l_y = 2  # y length

def n_to_ij(n, l_x):
    return (n%l_x, n//l_x)

def ij_to_n(i,j,l_x):
    return i + j*l_x

T_start = [ij_to_n(i, j, l_x) for i in range(l_x) for j in range(l_y)]

# north
Tn_end =   [ij_to_n(i, max(0, j-1), l_x) for i in range(l_x) for j in range(l_y)]
# south
Ts_end =   [ij_to_n(i, min(l_y-1, j+1), l_x) for i in range(l_x) for j in range(l_y)]
# east
Te_end =   [ij_to_n(min(l_x-1, i+1), j, l_x) for i in range(l_x) for j in range(l_y)]
# west
Tw_end =   [ij_to_n(max(0, i-1), j, l_x) for i in range(l_x) for j in range(l_y)]

Tn = sp.coo_matrix((np.ones(l_x*l_y), (T_start, Tn_end)), shape=(l_x*l_y, l_x*l_y))
Ts = sp.coo_matrix((np.ones(l_x*l_y), (T_start, Ts_end)), shape=(l_x*l_y, l_x*l_y))
Te = sp.coo_matrix((np.ones(l_x*l_y), (T_start, Te_end)), shape=(l_x*l_y, l_x*l_y))
Tw = sp.coo_matrix((np.ones(l_x*l_y), (T_start, Tw_end)), shape=(l_x*l_y, l_x*l_y))

def syst_input_fcn(m):
    if m == 'n':
        return 0
    elif m == 's':
        return 1
    elif m == 'e':
        return 2
    else:
        return 3
 
syst = MDP([Tn, Ts, Te, Tw], input_fcn=syst_input_fcn, input_name='direction',
                             output_name='s')

### Connect system and labels

In [4]:
# label1 is resolved at state 1
def connection1(s):
    if s == 1:
        return set([1])
    else:
        return set([0])

# label2 is resolved at state 5
def connection2(s):
    if s[0] == 5:
        return set([1])
    else:
        return set([0])    
    
pmdp1 = ProductMDP(syst, map1, connection1)
pmdp2 = ProductMDP(pmdp1, map2, connection2)

### Solve LTL problem on product system

In [5]:
formula = '( ( ! reg1 ) U safe1 ) & ( ( ! reg2 ) U safe2 ) & ( F target )'
dfs, init, final, _ = formula_to_mdp(formula)

def connection3(ss):
    s = ss[0][0]
    l1 = ss[0][1]
    l2 = ss[1]
    ret = []
    if s == 7:   # we want to reach state 7
        ret.append('target')
    if s == 2:   # state 2 is labeled by label1
        ret.append('reg1')
    if s == 6:   # state 6 is labeled by label2
        ret.append('reg2')
    if l1 == 'safe':
        ret.append('safe1')
    if l2 == 'safe':
        ret.append('safe2')
    return set( (tuple(ret),) )

print pmdp2
print dfs

final_prod = ProductMDP(pmdp2, dfs, connection3)
print final_prod

def accept(sss):
    return sss[1] in final

V, pol = final_prod.solve_reach(accept)

print V
print pol

MDP:  4 inputs "direction" --> 72 states "((s, label1), label2)"
MDP:  32 inputs "ap" --> 9 states "mu"
MDP:  4 inputs "direction" --> 648 states "(((s, label1), label2), mu)"
[ 0.25  1.    0.5   0.5   0.25  1.    0.5   0.5   0.    0.5   1.    0.5   1.
  0.5   1.    0.5   1.    0.    0.    1.    0.5   0.    0.    1.    0.5   0.
  0.    0.5   1.    1.    0.5   0.5   1.    1.    0.5   0.    1.    1.    1.
  1.    1.    1.    1.    1.    0.    0.    1.    1.    0.    0.    1.    1.
  0.    0.    0.    1.    0.    0.5   0.    1.    0.    0.5   0.    0.    1.
  0.    1.    0.    1.    0.    1.    0.    0.    1.    0.    0.    0.    1.
  0.    0.    0.    0.25  1.    0.5   0.5   0.25  1.    0.5   0.5   0.    0.5
  1.    0.5   1.    0.5   1.    0.5   1.    0.    0.    1.    0.5   0.    0.
  1.    0.5   0.    0.    0.5   1.    1.    0.5   0.5   1.    1.    0.5   0.
  1.    1.    1.    1.    1.    1.    1.    1.    0.    0.    1.    1.    0.
  0.    1.    1.    0.    0.    0.    1.    0.    0.5